In [ ]:
!python --version

Python 3.8.10


In [1]:
! pip install torch seaborn torchvision tensorflow biopython accelerate transformers datasets huggingface_hub evaluate requests pandas sklearn optuna ray[tune] hyperopt nvidia-ml-py3 wandb -qqq

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.1/3.1 MB 24.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 199.7/199.7 KB 20.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.3/6.3 MB 57.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 462.8/462.8 KB 14.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 190.3/190.3 KB 22.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 81.4/81.4 KB 9.7 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.4/57.4 MB 11.1 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 44.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.6/7.6 MB 102.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 132.0/132.0 KB 18.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 213.0/213.0 KB 28.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [2]:
model_checkpoint = "facebook/esm2_t6_8M_UR50D"
model_name = model_checkpoint.split("/")[-1]
#model_checkpoint = "facebook/esm2_t12_35M_UR50D"
#model_checkpoint = "facebook/esm2_t30_150M_UR50D"
#model_checkpoint = "facebook/esm2_t33_650M_UR50D"

In [16]:
import requests
import math
from io import BytesIO
import pandas as pd
import gc, torch
from accelerate import Accelerator
from Bio import SeqIO 
from Bio.SeqRecord import SeqRecord
import seaborn as sns
from transformers import AutoModelForSequenceClassification, TrainingArguments, Trainer, AutoTokenizer, logging
from sklearn.model_selection import train_test_split
from datasets import Dataset
from sklearn.utils.random import sample_without_replacement
gc.collect()
torch.cuda.empty_cache()
accelerator = Accelerator()
device = accelerator.device

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [5]:
tokenizer = AutoTokenizer.from_pretrained(model_checkpoint,use_fast=True)
model = AutoModelForSequenceClassification.from_pretrained(model_checkpoint, num_labels=2)
data_dir = '/content/drive/My Drive/Colab Notebooks/datasets/'
model_dir = '/content/drive/My Drive/Colab Notebooks/models/'
run_number=1
ratio=1

Some weights of the model checkpoint at facebook/esm2_t6_8M_UR50D were not used when initializing EsmForSequenceClassification: ['lm_head.bias', 'lm_head.dense.weight', 'lm_head.layer_norm.weight', 'lm_head.dense.bias', 'lm_head.layer_norm.bias', 'lm_head.decoder.weight']
- This IS expected if you are initializing EsmForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing EsmForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of EsmForSequenceClassification were not initialized from the model checkpoint at facebook/esm2_t6_8M_UR50D and are newly initialized: ['classifier.out_proj.weight', 'classifier.dense.weight', 'classifier.dense.bias', 'clas

In [6]:
for seq_record in SeqIO.parse(data_dir+"bontx.fasta", "fasta"):
  s=str(seq_record.seq)

In [19]:
def split(tox_seq,nottox_seq):
  sequences=tox_seq+nottox_seq
  labels = [1 for i in tox_seq]+[0 for i in nottox_seq]
  train_sequences, test_sequences, train_labels, test_labels = train_test_split(sequences, labels, test_size=0.25, shuffle=True)
  return train_sequences, test_sequences, train_labels, test_labels

def tokenize_seqs(train_sequences, test_sequences):
  train_tokens=tokenizer(train_sequences)
  test_tokens=tokenizer(test_sequences)
  return train_tokens, test_tokens

def save_seq_splits(train_sequences, test_sequences, train_labels, test_labels,data_dir,ratio,trial_number,run_number):
  traindf=pd.DataFrame({"sequences":train_sequences,"labels":train_labels})
  testdf=pd.DataFrame({"sequences":test_sequences,"labels":test_labels})
  traindf.to_csv(data_dir+f"train-r:{ratio}tn:{trial_number}rn:{run_number}.csv")
  testdf.to_csv(data_dir+f"test-r:{ratio}tn:{trial_number}rn:{run_number}.csv")

def from_token_splits(data_dir,ratio,trial_number,run_number):
  train_tokens = pd.read_csv(data_dir+f"train-r:{ratio}tn:{trial_number}rn:{run_number}.csv")["tokens"].tolist()
  test_tokens = pd.read_csv(data_dir+f"test-r:{ratio}tn:{trial_number}rn:{run_number}.csv")["tokens"].tolist()
  train_labels = pd.read_csv(data_dir+f"train-r:{ratio}tn:{trial_number}rn:{run_number}.csv")["labels"].tolist()
  test_labels = pd.read_csv(data_dir+f"test-r:{ratio}tn:{trial_number}rn:{run_number}.csv")["labels"].tolist()
  return train_tokens, test_tokens, train_labels, test_labels

def create_datasets(train_tokens, test_tokens, train_labels, test_labels):
  train_dataset = Dataset.from_dict(train_tokens)
  test_dataset = Dataset.from_dict(test_tokens)
  train_dataset = train_dataset.add_column("labels", train_labels)
  test_dataset = test_dataset.add_column("labels", test_labels)
  return train_dataset, test_dataset



def train_test_pipe(data_dir,ratio=1,trial_number=1,run_number=1,uniref_level=50,load_saved=False,output_class_lengths=False):
  if (load_saved==False):
    tox_seq=pd.read_csv(data_dir+f'tref{uniref_level}.csv')['Reference sequence'].tolist()
    nottox_seq=pd.read_csv(data_dir+f'ntref{uniref_level}-1trial-{ratio}:1-seq.csv')[f'trial{trial_number}'].tolist()
    train_sequences, test_sequences, train_labels, test_labels = split(tox_seq,nottox_seq)
    train_tokens, test_tokens = tokenize_seqs(train_sequences, test_sequences)
    save_seq_splits(train_sequences, test_sequences, train_labels, test_labels,data_dir,ratio,trial_number,run_number)
    train_dataset, test_dataset=create_datasets(train_tokens, test_tokens, train_labels, test_labels)
  elif (load_save==True):
    train_sequences, test_sequences, train_labels, test_labels = from_token_splits(data_dir,ratio,trial_number,run_number)
    train_tokens, test_tokens = tokenize_seqs(train_sequences, test_sequences)
    train_dataset, test_dataset=create_datasets(train_tokens, test_tokens, train_labels, test_labels)
  if (output_class_lengths==False):
    tox_seq=pd.read_csv(data_dir+f'tref{uniref_level}.csv')['Reference sequence'].tolist()
    nottox_seq=pd.read_csv(data_dir+f'ntref{uniref_level}-{ratio}:1-seq.csv')[f'trial{trial_number}'].tolist()
    seq_counts=pd.DataFrame([['not toxin',len(ntref50_labels)],['toxin',len(tref50_labels)]],columns=["sequence type","data count"])
    print(seq_counts)
  return train_dataset, test_dataset

In [20]:
train, test=train_test_pipe(data_dir=data_dir,ratio=ratio,trial_number=1,run_number=run_number,uniref_level=50,output_class_lengths=True)

In [22]:
%env WANDB_PROJECT=esm-fine-tuning-toxins
%env WANDB_NOTEBOOK_NAME=esm_fine_tuning_hyperparamsearch_2class
%env WANDB_WATCH=all

env: WANDB_PROJECT=esm-fine-tuning-toxins
env: WANDB_NOTEBOOK_NAME=esm_fine_tuning_hyperparamsearch_2class
env: WANDB_WATCH=all


In [21]:
import wandb
wandb.login()

ERROR:wandb.jupyter:Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.


<IPython.core.display.Javascript object>

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter, or press ctrl+c to quit: 

··········


wandb: Paste an API key from your profile and hit enter, or press ctrl+c to quit: 

··········


wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


True

In [23]:
from evaluate import load
import numpy as np

metric = load("accuracy")
trainingaccuracy=[]
def compute_metrics(eval_pred):
    predictions, labels = eval_pred
    predictions = np.argmax(predictions, axis=1)
    accuracy=metric.compute(predictions=predictions, references=labels)
    trainingaccuracy.append(accuracy)
    return accuracy

In [45]:
from transformers import TrainerCallback
class hyperparamcallback(TrainerCallback):
  def __init__(self, *args, **kwargs):
      super().__init__(*args, **kwargs)
      self.args=args
  def on_train_begin(self, args, state, control, **kwargs):
        run=f"{model_name}-{ratio}:1-run:{run_number}-{args.num_train_epochs}-{args.learning_rate}-{args.per_device_train_batch_size}"
        print("Starting training:"+run)
  def on_train_end(self, args, state, control, **kwargs):
        print("Ending training:"+run)

In [39]:
def model_init(trial):
    return AutoModelForSequenceClassification.from_pretrained(model_checkpoint,num_labels=2,return_dict=True)

In [51]:
training_args = TrainingArguments(
    overwrite_output_dir =True,
    evaluation_strategy = 'steps',
    eval_steps=500,
    warmup_ratio=0.01,
    save_strategy="steps",
    save_steps=1000,
    save_total_limit=5,
    learning_rate=2e-5,
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    num_train_epochs=3,
    fp16=True,
    weight_decay=0.01,
    run_name=f"{model_name}",
    output_dir=f"/content/drive/My Drive/Colab Notebooks/models/hyperopt2class/{model_name}",
    report_to=['wandb'],
    load_best_model_at_end=True,
    metric_for_best_model="accuracy",
    push_to_hub=False,
)

PyTorch: setting up devices


In [52]:
class CustomTrainer(Trainer):
    def __init__(self, *args, **kwargs):
        super().__init__(*args, **kwargs)
        self.args.run_name=f"{model_name}-{ratio}:1-run:{run_number}-{self.args.num_train_epochs}-{self.args.learning_rate}-{self.args.per_device_train_batch_size}"
        self.args.output_dir=f"/content/drive/My Drive/Colab Notebooks/models/hyperopt2class/{model_name}-{ratio}:1-run:{run_number}-{self.args.num_train_epochs}-{self.args.learning_rate}-{self.args.per_device_train_batch_size}"

In [57]:
from transformers import DataCollatorWithPadding
from transformers import EarlyStoppingCallback
#EarlyStoppingCallback(early_stopping_patience=3,early_stopping_threshold=0.02)
trainer = CustomTrainer(
    args=training_args,
    train_dataset=train,
    eval_dataset=test,
    compute_metrics=compute_metrics,
    tokenizer=tokenizer,
    model_init=model_init,
    #data_collator=DataCollatorWithPadding,
    callbacks = [hyperparamcallback]

)

loading configuration file config.json from cache at /root/.cache/huggingface/hub/models--facebook--esm2_t6_8M_UR50D/snapshots/97bbd7b3dc8acb222027cde0d8e911f3de78d5b8/config.json
Model config EsmConfig {
  "_name_or_path": "facebook/esm2_t6_8M_UR50D",
  "architectures": [
    "EsmForMaskedLM"
  ],
  "attention_probs_dropout_prob": 0.0,
  "classifier_dropout": null,
  "emb_layer_norm_before": false,
  "esmfold_config": null,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.0,
  "hidden_size": 320,
  "initializer_range": 0.02,
  "intermediate_size": 1280,
  "is_folding_model": false,
  "layer_norm_eps": 1e-05,
  "mask_token_id": 32,
  "max_position_embeddings": 1026,
  "model_type": "esm",
  "num_attention_heads": 20,
  "num_hidden_layers": 6,
  "pad_token_id": 1,
  "position_embedding_type": "rotary",
  "token_dropout": true,
  "torch_dtype": "float32",
  "transformers_version": "4.26.1",
  "use_cache": true,
  "vocab_list": null,
  "vocab_size": 33
}

loading weights file pytorch_mo

In [54]:
def searchspace(trial):
    from ray import tune

    return {
        "learning_rate": tune.loguniform(2e-5, 2e-2),
        "num_train_epochs": tune.choice(range(1, 6)),
        "seed": tune.choice(range(1, 41)),
        "per_device_train_batch_size": tune.choice([4, 8, 16, 32, 64]),
        #"train_dataset": tune.choice(["1:1class_train","2:1class_train","5:1class_train","10:1class_train"]),
        #"eval_dataset" : "train_dataset"[0:-5]+"test"
    }

In [60]:
!pip install optuna

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 365.3/365.3 KB 12.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 210.6/210.6 KB 21.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.7/78.7 KB 8.6 MB/s eta 0:00:00


In [58]:
def hp_space(trial):
    return {
        "learning_rate": trial.suggest_float("learning_rate", 2e-5, 1e-2, log=True),
        "num_train_epochs": trial.suggest_int("num_train_epochs", 1, 6),
        "seed": trial.suggest_int("seed", 1, 40),
        "per_device_train_batch_size": trial.suggest_categorical("per_device_train_batch_size", [4, 8, 16, 32, 64]),
    }

In [ ]:
best_trial = trainer.hyperparameter_search(
    direction="maximize",
    backend="optuna",
    hp_space=hp_space,
    n_trials=5,
)

[I 2023-02-14 03:10:56,302] A new study created in memory with name: no-name-adfc5166-0abe-4817-8fcf-3e791c151581
Trial: {'learning_rate': 0.000307877301998827, 'num_train_epochs': 2, 'seed': 18, 'per_device_train_batch_size': 4}
loading configuration file config.json from cache at /root/.cache/huggingface/hub/models--facebook--esm2_t6_8M_UR50D/snapshots/97bbd7b3dc8acb222027cde0d8e911f3de78d5b8/config.json
Model config EsmConfig {
  "_name_or_path": "facebook/esm2_t6_8M_UR50D",
  "architectures": [
    "EsmForMaskedLM"
  ],
  "attention_probs_dropout_prob": 0.0,
  "classifier_dropout": null,
  "emb_layer_norm_before": false,
  "esmfold_config": null,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.0,
  "hidden_size": 320,
  "initializer_range": 0.02,
  "intermediate_size": 1280,
  "is_folding_model": false,
  "layer_norm_eps": 1e-05,
  "mask_token_id": 32,
  "max_position_embeddings": 1026,
  "model_type": "esm",
  "num_attention_heads": 20,
  "num_hidden_layers": 6,
  "pad_token_id

wandb: Currently logged in as: cchallacombe (biolm). Use `wandb login --relogin` to force relogin


Starting training:esm2_t6_8M_UR50D-1:1-run:1-2-0.000307877301998827-4


Step,Training Loss,Validation Loss,Accuracy
500,0.061400,0.000242,1.000000
1000,0.000100,0.000048,1.000000
1500,0.000000,0.000022,1.000000
2000,0.000000,0.000012,1.000000


***** Running Evaluation *****
  Num examples = 36301
  Batch size = 8
***** Running Evaluation *****
  Num examples = 36301
  Batch size = 8
Saving model checkpoint to /content/drive/My Drive/Colab Notebooks/models/hyperopt2class/esm2_t6_8M_UR50D-1:1-run:1-3-2e-05-8/run-0/checkpoint-1000
Configuration saved in /content/drive/My Drive/Colab Notebooks/models/hyperopt2class/esm2_t6_8M_UR50D-1:1-run:1-3-2e-05-8/run-0/checkpoint-1000/config.json
Model weights saved in /content/drive/My Drive/Colab Notebooks/models/hyperopt2class/esm2_t6_8M_UR50D-1:1-run:1-3-2e-05-8/run-0/checkpoint-1000/pytorch_model.bin
tokenizer config file saved in /content/drive/My Drive/Colab Notebooks/models/hyperopt2class/esm2_t6_8M_UR50D-1:1-run:1-3-2e-05-8/run-0/checkpoint-1000/tokenizer_config.json
Special tokens file saved in /content/drive/My Drive/Colab Notebooks/models/hyperopt2class/esm2_t6_8M_UR50D-1:1-run:1-3-2e-05-8/run-0/checkpoint-1000/special_tokens_map.json
***** Running Evaluation *****
  Num example

In [ ]:
trainer.evaluate()

***** Running Evaluation *****
  Num examples = 69859
  Batch size = 8


{'eval_loss': 0.3635565936565399,
 'eval_accuracy': 0.8840664767603316,
 'eval_runtime': 283.5383,
 'eval_samples_per_second': 246.383,
 'eval_steps_per_second': 30.8,
 'epoch': 3.0}

In [ ]:
model_dir = '/content/drive/My Drive/Colab Notebooks/models/'

In [ ]:
trainer.save_model(model_dir + 'esm_fine_tuning_10epochs_trainer_earlystopping')
model.save_pretrained(model_dir + 'esm_fine_tuning_10epochs_model_earlystopping')

Saving model checkpoint to /content/drive/My Drive/Colab Notebooks/models/esm_fine_tuning3epochstrainer
Configuration saved in /content/drive/My Drive/Colab Notebooks/models/esm_fine_tuning3epochstrainer/config.json
Model weights saved in /content/drive/My Drive/Colab Notebooks/models/esm_fine_tuning3epochstrainer/pytorch_model.bin
tokenizer config file saved in /content/drive/My Drive/Colab Notebooks/models/esm_fine_tuning3epochstrainer/tokenizer_config.json
Special tokens file saved in /content/drive/My Drive/Colab Notebooks/models/esm_fine_tuning3epochstrainer/special_tokens_map.json
Configuration saved in /content/drive/My Drive/Colab Notebooks/models/esm_fine_tuning3epochsmodel/config.json
Model weights saved in /content/drive/My Drive/Colab Notebooks/models/esm_fine_tuning3epochsmodel/pytorch_model.bin


In [ ]:
wandb.finish()

eval/accuracy,▁███
eval/loss,▁▃█▃
eval/runtime,▁▃▅█
eval/samples_per_second,█▆▄▁
eval/steps_per_second,█▆▄▁
train/epoch,▁▁▁▂▂▂▂▂▂▃▃▃▃▃▃▄▄▄▄▄▅▅▅▅▅▆▆▆▆▆▆▇▇▇▇▇████
train/global_step,▁▁▁▂▂▂▂▂▂▃▃▃▃▃▃▄▄▄▄▄▅▅▅▅▅▆▆▆▆▆▆▇▇▇▇▇▇███
train/learning_rate,████▇▇▇▇▇▆▆▆▆▆▆▅▅▅▅▅▄▄▄▄▄▄▃▃▃▃▃▂▂▂▂▂▂▁▁▁
train/loss,▇█▇▇▇▆▆▆▆▆▆▆▅▅▄▃▄▃▄▄▄▃▅▄▄▄▃▁▂▂▂▂▂▁▃▁▂▂▁▂
train/total_flos,▁
train/train_loss,▁


In [ ]:
model = AutoModelForSequenceClassification.from_pretrained(model_dir + 'esm_fine_tuning3epochsmodel', num_labels=2)
tokenizer = AutoTokenizer.from_pretrained(model_checkpoint)

In [ ]:
bontx_encoded=tokenizer(s,return_tensors='pt')
bontx_output=model(**bontx_encoded)

In [ ]:
from torch import nn
probabilities = nn.functional.softmax(bontx_output.logits, dim=-1)
print(probabilities)

tensor([[0.0655, 0.9345]], grad_fn=<SoftmaxBackward0>)
